# Reconhecimento de veículos na estrada

In [1]:
import math
import cv2

In [2]:
class EuclideanDistTracker:
    def __init__(self):
        # Store the center of each object
        # key = object id
        # value = center coordinates
        self.centers = {}

        # ID counter
        self.id_count = 0

    def update(self, objects_rect):
        # Tuple that stores the objects coordinates and id
        objects = []

        # Get center point of new object
        for rect in objects_rect:
            # Getting rectangle properties
            x, y, w, h = rect
            x_center = (2 * x + w) // 2
            y_center = (2 * y + h) // 2

            # Flag for repeated object
            same_object = False             

            # Iterating over all points
            for id, pt in self.centers.items():

                # Calculating Euclidean distance
                dist = math.hypot(x_center - pt[0], y_center - pt[1])

                # Check the distance
                if dist < 25:
                    self.centers[id] = (x_center, y_center)
                    print(f"Objects: {self.centers}")
                    objects.append((x, y, w, h, id))
                    
                    # Setting same_object so we keep the original ID
                    same_object = True
                    break

            # Assign an ID for the new object
            if not same_object:
                self.centers[self.id_count] = (x_center, y_center)
                objects.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Cleaning up the dictionary to remove IDs out of frame
        new_centers = {}
        for obj in objects:
            object_id = obj[4]
            center = self.centers[object_id]
            new_centers[object_id] = center

        # Update dictionary with IDs not used removed
        self.centers = new_centers.copy()
        return objects

In [3]:
# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("highway.mp4")

# Object detection
detector = cv2.createBackgroundSubtractorMOG2(
    history=100, 
    varThreshold=40
)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract the ROI
    roi = frame[340:720, 500:800]

    # Apply our detector in the ROI
    mask = detector.apply(roi)

    # Thresholding to remove the white generated by the shadow
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)

    # Finding contours
    contours, _ = cv2.findContours(
        mask, 
        cv2.RETR_TREE, 
        cv2.CHAIN_APPROX_SIMPLE
    )

    # Store all detections
    detections = []

    # Iterating over contours
    for cnt in contours:
        # Calculate contour area
        area = cv2.contourArea(cnt)                 

        # Discarding small objects
        if area > 100:
            # Create a rectangle
            x, y, w, h = cv2.boundingRect(cnt)      
            detections.append((x, y, w, h))         

    # Updating the IDs using the tracker
    objects = tracker.update(detections)
    for obj in objects:
        x, y, w, h, id = obj
        cv2.putText(
            roi,
            str(id),
            (x, y - 15),
            cv2.FONT_HERSHEY_PLAIN,
            2,
            (255, 0, 0),
            2
        )
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

    cv2.imshow("Region Of Interest", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(30)
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

Objects: {0: (163, 79)}
Objects: {1: (161, 122)}
Objects: {1: (163, 127)}
Objects: {1: (162, 136)}
Objects: {1: (163, 149)}
Objects: {1: (160, 144)}
Objects: {1: (159, 153)}
Objects: {1: (162, 174)}
Objects: {1: (160, 168), 2: (158, 148)}
Objects: {1: (159, 180)}
Objects: {1: (158, 190)}
Objects: {1: (158, 198)}
Objects: {1: (158, 210)}
Objects: {1: (157, 220)}
Objects: {1: (156, 232)}
Objects: {1: (156, 244)}
Objects: {1: (156, 256)}
Objects: {1: (155, 269)}
Objects: {1: (154, 284)}
Objects: {1: (153, 298)}
Objects: {1: (152, 314)}
Objects: {1: (151, 330)}
Objects: {1: (150, 342)}
Objects: {1: (148, 351)}
Objects: {1: (145, 360)}
Objects: {1: (143, 369)}
Objects: {3: (207, 20)}
Objects: {3: (207, 24)}
Objects: {3: (208, 26)}
Objects: {3: (208, 30)}
Objects: {3: (208, 33)}
Objects: {3: (209, 28)}
Objects: {3: (208, 40)}
Objects: {3: (208, 35)}
Objects: {3: (209, 38)}
Objects: {3: (209, 41)}
Objects: {3: (208, 53)}
Objects: {3: (209, 48)}
Objects: {3: (210, 52)}
Objects: {3: (210, 64)}
